In [21]:
import pandas as pd
import codecs
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from collections import Counter

In [22]:
newspapers = pd.read_csv("C:/Users/jw156/Ironhack/vaccine/vaccine_sentiment/english/covid_newspapers.csv", index_col=0)

In [23]:
newspapers

,title,href,date,new_title,id,corona_score,vaccine
1,guardian,https://www.theguardian.com/politics/2021/mar/...,2021-03-19,bori johnson receiv oxfordastrazeneca covid va...,politics,1.000000,NaN
3,guardian,https://www.theguardian.com/world/2021/feb/19/...,2021-02-19,sceptic oxford vaccin threaten europ s immunis...,world,1.000000,NaN
4,guardian,https://www.theguardian.com/world/2021/mar/18/...,2021-03-18,thursday brief EU s expert give oxford vaccin ...,world,1.000000,NaN
5,guardian,https://www.theguardian.com/world/2021/mar/16/...,2021-03-16,oxfordastrazeneca vaccin countri paus jab,world,1.000000,oxfordastrazeneca
6,guardian,https://www.theguardian.com/world/2021/mar/16/...,2021-03-16,chao germani itali suspens oxford vaccin,world,1.000000,NaN
...,...,...,...,...,...,...,...
32023,independent,https://www.independent.co.uk/news/education/e...,2021-03-25,not enough evid stop wear mask secondari schoo...,news,0.538462,NaN
32024,independent,https://www.independent.co.uk/news/world/ameri...,2021-03-25,brett favr reveal almost want kill amid painki...,news,0.555556,NaN
32025,independent,https://www.independent.co.uk/news/reports-say...,2021-03-25,report say cnn s chri cuomo got special covid ...,news,1.000000,NaN
32026,independent,https://www.independent.co.uk/news/uk/home-new...,2021-03-25,univers aberdeen becom one first UK institut r...,news,0.600000,NaN


In [24]:
newspapers.set_index("href", inplace=True)

In [25]:
newspapers = newspapers.reset_index(drop=True)

In [26]:
newspapers.title.drop_duplicates(inplace=True)

In [27]:
newspapers

,title,date,new_title,id,corona_score,vaccine
0,guardian,2021-03-19,bori johnson receiv oxfordastrazeneca covid va...,politics,1.000000,NaN
1,guardian,2021-02-19,sceptic oxford vaccin threaten europ s immunis...,world,1.000000,NaN
2,guardian,2021-03-18,thursday brief EU s expert give oxford vaccin ...,world,1.000000,NaN
3,guardian,2021-03-16,oxfordastrazeneca vaccin countri paus jab,world,1.000000,oxfordastrazeneca
4,guardian,2021-03-16,chao germani itali suspens oxford vaccin,world,1.000000,NaN
...,...,...,...,...,...,...
68403,independent,2021-03-25,not enough evid stop wear mask secondari schoo...,news,0.538462,NaN
68404,independent,2021-03-25,brett favr reveal almost want kill amid painki...,news,0.555556,NaN
68405,independent,2021-03-25,report say cnn s chri cuomo got special covid ...,news,1.000000,NaN
68406,independent,2021-03-25,univers aberdeen becom one first UK institut r...,news,0.600000,NaN


# POS Determination of words

In [28]:
newspapers = newspapers.reset_index(drop=True)

In [29]:
import nltk
from nltk.stem import WordNetLemmatizer 

In [30]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize


In [31]:
from nltk.corpus import wordnet as wn

def pos_headline(headline):
    new_headline = headline.split(' ')
    word_list = []
    for word in new_headline:
        if word == None or word.isalpha()==False:
            continue
        else:
            text = word_tokenize(word)
            postition_tag = nltk.pos_tag(text)
            word_list.append(postition_tag)
    return word_list

def is_proper_noun(tag):
    return tag in ['NNP', 'NNPS']

def is_noun(tag):
    return tag in ['NN', 'NNS']


def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']


def is_adverb(tag):
    return tag in ['RB', 'RBR', 'RBS']


def is_adjective(tag):
    return tag in ['JJ', 'JJR', 'JJS']



def penn_to_wn(headlines):
    headline_list = pos_headline(headlines)
    tagged_list = []
    for tag in headline_list:
        tag = list(tag[0])
        if is_adjective(tag[1]):
            tag[1] = (wn.ADJ)
            tagged_list.append(tag)
        elif is_noun(tag[1]):
            tag[1] =(wn.NOUN)
            tagged_list.append(tag)
        elif is_adverb(tag[1]):
            tag[1] =(wn.ADV)
            tagged_list.append(tag)
        elif is_verb(tag[1]):
            tag[1] =(wn.VERB)
            tagged_list.append(tag)
        elif is_proper_noun(tag[1]):
            tag[1] =('NNP')
            tagged_list.append(tag)
    return tagged_list

def lemming_words(headline):
    if len(headline) == 0:
        return 0
    headline_list = penn_to_wn(headline)    
    new_string = []
    lemmatizer = WordNetLemmatizer()
    for word in headline_list:
        if word[1] == 'NNP':
            continue
        else:
            word[0] = (lemmatizer.lemmatize(word[0], word[1]))
            new_string.append(word)

    return new_string

In [32]:
x = newspapers.new_title[1]
lemming_words(x)


[['sceptic', 'a'],
 ['oxford', 'n'],
 ['vaccin', 'n'],
 ['threaten', 'n'],
 ['europ', 'n'],
 ['s', 'n'],
 ['immunis', 'n'],
 ['push', 'n']]

In [33]:
newspapers['pos_headline'] = newspapers['new_title'].apply(pos_headline)

In [34]:
newspapers['lem_pos_headline'] = newspapers['new_title'].apply(lambda x: lemming_words(x))

In [35]:
newspapers['pos_headline'][1]

[[('sceptic', 'JJ')],
 [('oxford', 'NN')],
 [('vaccin', 'NN')],
 [('threaten', 'NNS')],
 [('europ', 'NN')],
 [('s', 'NN')],
 [('immunis', 'NN')],
 [('push', 'NN')]]

# Word Sense Disambiguation

In [36]:
from nltk.wsd import lesk

x = 'The porn threatens to cause a lockdown'

print(lesk(x.split(), 'threatens', 'v'))


Synset('threaten.v.03')


In [37]:
def word_definitions(title, lemmed_title):
    synsets = []
    original_text = title
    lemmed_pos = lemmed_title
    for i in lemmed_pos:
        synsets.append(lesk(original_text.split(), i[0], i[1]))
    return synsets

In [39]:
# x = newspapers['title'][1]
# y = newspapers['lem_pos_headline'][1]
# import re
# from nltk.corpus import sentiwordnet as swn,SentiSynset

# j = word_definitions(x, y)
# k = str(j[0])
# print(k)
# x = (re.findall(r"'([^']*)'", k))
# print(x)
# f = swn.senti_synset(x[0])
# print(f)

In [40]:
newspapers['word_definitions'] = newspapers.apply(lambda x: word_definitions(x['title'], x['lem_pos_headline']), axis=1)


In [41]:
x = newspapers['word_definitions'][1]

In [42]:
for i in x:
    syn_word = (re.findall(r"'([^']*)'", str(i)))
    if len(syn_word) != 0:
        input_word = (syn_word[0])
        word_senti = (swn.senti_synset(input_word))
        pos = float(word_senti.pos_score()) 
        neg = float(word_senti.neg_score())
        overall_senti = pos + neg
        print(overall_senti)

0.0
0.0
0.0


## Save dataframe

In [43]:
# newspapers.to_csv('C:/Users/jw156/Ironhack/vaccine/newspaper_pos_score.csv')
# print("Sucessfully saved")

## Load dataframe

In [44]:
newspapers = pd.read_csv('C:/Users/jw156/Ironhack/vaccine/newspaper_pos_score.csv')
newspapers.set_index("hrefs", inplace=True)

In [45]:
newspapers

,title,new_title,id,date,newspaper_name,vaccine,pos_headline,lem_pos_headline,word_definitions
hrefs,,,,,,,,,
https://www.theguardian.com/politics/2021/mar/19/boris-johnson-receives-first-dose-of-oxfordastrazenca-covid-vaccine,Boris Johnson receives Oxford/AstraZeneca Covi...,bor johnson receiv oxfordastrazenec covid vaccin,politics,2021-03-19,guardian,NaN,"[[('Boris', 'NNP')], [('Johnson', 'NNP')], [('...","[['receives', 'n'], ['Covid', 'n'], ['vaccine'...","[None, None, Synset('vaccine.n.01')]"
https://www.theguardian.com/world/2021/feb/19/german-politicians-counter-astrazeneca-covid-vaccine-scepticism-with-show-of-support,Scepticism over Oxford vaccine threatens Europ...,sceiv oxford vaccin threatens europ immun push,world,2021-02-19,guardian,NaN,"[[('Scepticism', 'NN')], [('over', 'IN')], [('...","[['Scepticism', 'n'], ['Oxford', 'n'], ['vacci...","[Synset('agnosticism.n.02'), Synset('oxford.n...."
https://www.theguardian.com/world/2021/mar/18/thursday-briefing-eus-experts-to-give-oxford-vaccine-verdict,Thursday briefing: EU's experts to give Oxford...,thursday brief eu expert giv oxford vaccin ver...,world,2021-03-18,guardian,NaN,"[[('Thursday', 'NNP')], [('experts', 'NNS')], ...","[['expert', 'n'], ['give', 'v'], ['Oxford', 'n...","[Synset('expert.n.01'), Synset('give.v.18'), S..."
https://www.theguardian.com/world/2021/mar/16/oxford-astrazeneca-vaccine-update-pausing-jabs,Oxford/AstraZeneca vaccine: which countries ha...,oxfordastrazenec vaccin country paus jab,world,2021-03-16,guardian,NaN,"[[('which', 'WDT')], [('countries', 'NNS')], [...","[['country', 'n'], ['have', 'v'], ['pause', 'v...","[Synset('country.n.04'), Synset('take.v.35'), ..."
https://www.theguardian.com/world/2021/mar/16/chaos-in-germany-and-italy-after-suspension-of-oxford-vaccine,Chaos in Germany and Italy after suspension of...,chao germany ita suspend oxford vaccin,world,2021-03-16,guardian,NaN,"[[('Chaos', 'NN')], [('in', 'IN')], [('Germany...","[['Chaos', 'n'], ['suspension', 'n'], ['Oxford...","[Synset('chaos.n.02'), Synset('suspension.n.05..."
...,...,...,...,...,...,...,...,...,...
https://www.independent.co.uk/news/uk/politics/lockdown-law-extension-coronavirus-act-b1822637.html,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[('Extreme', 'NN')], [('lockdown', 'NN')], [(...","[['Extreme', 'n'], ['lockdown', 'n'], ['law', ...","[Synset('extreme_point.n.01'), Synset('lockdow..."
https://www.independent.co.uk/news/uk/politics/lockdown-law-extension-coronavirus-act-b1822511.html,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[('Extreme', 'NN')], [('lockdown', 'NN')], [(...","[['Extreme', 'n'], ['lockdown', 'n'], ['law', ...","[Synset('extreme_point.n.01'), Synset('lockdow..."
https://www.independent.co.uk/independentpremium/business/covid-vaccine-pubs-boris-johnson-b1822508.html,The problem with Johnson’s vaccine passports f...,the problem johnson vaccin passport pub,business,2021-03-25,independent,NaN,"[[('The', 'DT')], [('problem', 'NN')], [('with...","[['problem', 'n'], ['vaccine', 'n'], ['passpor...","[Synset('problem.n.02'), Synset('vaccine.n.01'..."


# TF − IDF ∗ SentimentScore

In [46]:
newspapers

,title,new_title,id,date,newspaper_name,vaccine,pos_headline,lem_pos_headline,word_definitions
hrefs,,,,,,,,,
https://www.theguardian.com/politics/2021/mar/19/boris-johnson-receives-first-dose-of-oxfordastrazenca-covid-vaccine,Boris Johnson receives Oxford/AstraZeneca Covi...,bor johnson receiv oxfordastrazenec covid vaccin,politics,2021-03-19,guardian,NaN,"[[('Boris', 'NNP')], [('Johnson', 'NNP')], [('...","[['receives', 'n'], ['Covid', 'n'], ['vaccine'...","[None, None, Synset('vaccine.n.01')]"
https://www.theguardian.com/world/2021/feb/19/german-politicians-counter-astrazeneca-covid-vaccine-scepticism-with-show-of-support,Scepticism over Oxford vaccine threatens Europ...,sceiv oxford vaccin threatens europ immun push,world,2021-02-19,guardian,NaN,"[[('Scepticism', 'NN')], [('over', 'IN')], [('...","[['Scepticism', 'n'], ['Oxford', 'n'], ['vacci...","[Synset('agnosticism.n.02'), Synset('oxford.n...."
https://www.theguardian.com/world/2021/mar/18/thursday-briefing-eus-experts-to-give-oxford-vaccine-verdict,Thursday briefing: EU's experts to give Oxford...,thursday brief eu expert giv oxford vaccin ver...,world,2021-03-18,guardian,NaN,"[[('Thursday', 'NNP')], [('experts', 'NNS')], ...","[['expert', 'n'], ['give', 'v'], ['Oxford', 'n...","[Synset('expert.n.01'), Synset('give.v.18'), S..."
https://www.theguardian.com/world/2021/mar/16/oxford-astrazeneca-vaccine-update-pausing-jabs,Oxford/AstraZeneca vaccine: which countries ha...,oxfordastrazenec vaccin country paus jab,world,2021-03-16,guardian,NaN,"[[('which', 'WDT')], [('countries', 'NNS')], [...","[['country', 'n'], ['have', 'v'], ['pause', 'v...","[Synset('country.n.04'), Synset('take.v.35'), ..."
https://www.theguardian.com/world/2021/mar/16/chaos-in-germany-and-italy-after-suspension-of-oxford-vaccine,Chaos in Germany and Italy after suspension of...,chao germany ita suspend oxford vaccin,world,2021-03-16,guardian,NaN,"[[('Chaos', 'NN')], [('in', 'IN')], [('Germany...","[['Chaos', 'n'], ['suspension', 'n'], ['Oxford...","[Synset('chaos.n.02'), Synset('suspension.n.05..."
...,...,...,...,...,...,...,...,...,...
https://www.independent.co.uk/news/uk/politics/lockdown-law-extension-coronavirus-act-b1822637.html,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[('Extreme', 'NN')], [('lockdown', 'NN')], [(...","[['Extreme', 'n'], ['lockdown', 'n'], ['law', ...","[Synset('extreme_point.n.01'), Synset('lockdow..."
https://www.independent.co.uk/news/uk/politics/lockdown-law-extension-coronavirus-act-b1822511.html,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[('Extreme', 'NN')], [('lockdown', 'NN')], [(...","[['Extreme', 'n'], ['lockdown', 'n'], ['law', ...","[Synset('extreme_point.n.01'), Synset('lockdow..."
https://www.independent.co.uk/independentpremium/business/covid-vaccine-pubs-boris-johnson-b1822508.html,The problem with Johnson’s vaccine passports f...,the problem johnson vaccin passport pub,business,2021-03-25,independent,NaN,"[[('The', 'DT')], [('problem', 'NN')], [('with...","[['problem', 'n'], ['vaccine', 'n'], ['passpor...","[Synset('problem.n.02'), Synset('vaccine.n.01'..."


In [55]:
N = len (newspapers)
processed_text = []

for i in newspapers.new_title[:N]:
    processed_text.append(i)

In [56]:
DF = {}

for i in range(N):
    tokens = processed_text[i]
    tokens = tokens.split(' ')
    for w in tokens:
        w = w.lower()
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [57]:
total_vocab_size = len(DF)

In [58]:
total_vocab_size

15908

In [59]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [60]:
doc_freq('sceptic')

0

In [65]:
doc = 0
tf_idf = {}
for i in range(N):
    headline = processed_text[i]
    new_headline = headline.split(' ')
    counter = Counter(new_headline)
    words_count = len(new_headline)
    for token in np.unique(new_headline):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log(N/(df+1))
        tf_idf[doc, token] = tf*idf
    doc += 1

In [66]:
tf_idf

{(0, 'bor'): 0.489540321620454,
 (0, 'covid'): 0.38862831940317755,
 (0, 'johnson'): 0.477609520695824,
 (0, 'oxfordastrazenec'): 1.194012612498484,
 (0, 'receiv'): 0.9387665506710859,
 (0, 'vaccin'): 0.4598985027374064,
 (1, 'europ'): 0.6324680086575296,
 (1, 'immun'): 0.7679706385913105,
 (1, 'oxford'): 0.7184176466138661,
 (1, 'push'): 0.7888354199619731,
 (1, 'sceiv'): 1.0295193270585286,
 (1, 'threatens'): 0.8394601980977523,
 (1, 'vaccin'): 0.39419871663206263,
 (2, 'brief'): 0.5211732013769196,
 (2, 'eu'): 0.5275917235798546,
 (2, 'expert'): 0.5312111301113582,
 (2, 'giv'): 0.541204698879849,
 (2, 'oxford'): 0.6286154407871328,
 (2, 'thursday'): 0.7599312035371918,
 (2, 'vaccin'): 0.34492387705305483,
 (2, 'verdict'): 1.0328359954573767,
 (3, 'country'): 0.8448064614595003,
 (3, 'jab'): 1.0077649194559548,
 (3, 'oxfordastrazenec'): 1.432815134998181,
 (3, 'paus'): 1.3661168133015658,
 (3, 'vaccin'): 0.5518782032848878,
 (4, 'chao'): 0.8971312509383573,
 (4, 'germany'): 0.8628905

In [12]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [13]:
DF['i']

1445

In [14]:
processed_text = ['when harry met sally', 'I like big noses']
N = len(processed_text)
from collections import Counter

doc = 0
tf_idf = {}
for i in range(N):
    headline = processed_text[i]
    new_headline = headline.split(' ')
    counter = Counter(new_headline)
    words_count = len(new_headline)
    for token in np.unique(new_headline):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        tf_idf[doc, token] = tf*idf
    doc += 1 

In [15]:
N

2

In [16]:
index_df=list(df_model2.index)


NameError: name 'df_model2' is not defined

In [17]:
N = len (df_model2)

sum_sentiment={}

for index in index_df:
    headline = processed_text[index]
    new_headline = headline.split(' ')
    #print(new_headline)
    for token in np.unique(new_headline):
        #print(index)
        #print(token)
        if index in list(sum_sentiment.keys()):
            sentiment_value=float(tf_idf[(index, token)])*float(tf_idf[(index, token)])
            sum_sentiment[index] += sentiment_value
        else:
            sum_sentiment[index]=float(tf_idf[(index, token)])*float(tf_idf[(index, token)])

NameError: name 'df_model2' is not defined

In [ ]:
sum_sentiment

In [ ]:
df_model2["sentiment"]=df_model2.apply(lambda row: sum_sentiment[row.name],axis=1)

In [ ]:
df_model2